In [1]:
import os
import codecs
import pandas as pd
import numpy as np
import warnings
import collections

# change dir for custom imports
os.chdir('../')
dataset = 'epinions'
dataset_path = os.path.join('datasets', dataset, 'v1')

In [ ]:
from numpy import array

# Read the data from the original epinions txt file
f = open(os.path.join(dataset_path, 'rating.txt'), 'r') # open the file for reading
data = []
for row_num, line in enumerate(f):
    values = line.strip().split('\t')
    data.append([v for v in values])

    if row_num == 9000000:
        break
all_data = array(data)
f.close()

## Load Data (If previously computed, skip this and load the file)

In [ ]:
from numpy import array

# Read the data from the original epinions txt file
f = open(os.path.join(dataset_path, 'rating.txt'), 'r') # open the file for reading
data = []
for row_num, line in enumerate(f):
    values = line.strip().split('\t')
    data.append([v for v in values])

    if row_num == 9000000:
        break
all_data = array(data)
f.close()

# Update the datatypes
ratings_df = pd.DataFrame(all_data).rename(columns={
    0: 'itemID',
    1: 'userID',
    2: 'rating',
    3: 'status'
})
# Convert multiple columns
ratings_df = ratings_df.astype({'itemID':'int64', 'userID':'int64', 'status':'int', 'rating':'int'})

# Dataset stats
print(f"Total dataset users: {len(set(ratings_df.userID.to_list()))}")
print(f"Total dataset ratings: {len(ratings_df.userID.to_list())}")

# Save the original dataset (to match the users selected with the social aspect in the upcoming experiment)
# ratings_df.to_csv('./datasets/epinions/v1/ratings_raw.csv', index=False)

In [2]:
ratings_df = pd.read_csv('./datasets/epinions/v1/ratings_raw.csv')

In [3]:
ratings_df.head()

,itemID,userID,rating,status,4,5,6,7
0,139431556,591156,5,0,2001/01/10,NaN,1,2518365
1,139431556,1312460676,5,0,2001/01/10,NaN,1,2518365
2,139431556,204358,5,0,2001/01/10,NaN,1,2518365
3,139431556,368725,5,0,2001/01/10,NaN,1,2518365
4,139431556,277629,5,0,2001/01/10,NaN,1,2518365


### Apply dataset filtering and create the features for Clustering (If pre-computed, just load the file)

In [ ]:
# Filter records that have at least I total ratings and U total users
I = 150
U = 100

# Items filter
grouped_item_ratings = ratings_df.groupby(by="itemID")["rating"].count()
selected_items = grouped_item_ratings[grouped_item_ratings > I].index.tolist()
df = ratings_df[ratings_df['itemID'].isin(selected_items)]

# Users filter
grouped_user_ratings = df.groupby(by="userID")["rating"].count()
selected_users = grouped_user_ratings[grouped_user_ratings > U].index.tolist()
df = df[df['userID'].isin(selected_users)]

# reset numbers to avoid long values and clean columns
df['userID2'] = pd.factorize(df['userID'])[0]
df['itemID2'] = pd.factorize(df['itemID'])[0]
df.drop(['status', 4, 5, 6, 7], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df = df[['userID2', 'itemID2', 'rating']].rename(columns={'userID2': 'userID', 'itemID2': 'itemID'})

# Dataset stats
print(f"Total dataset users: {len(set(df.userID.to_list()))}")
print(f"Total dataset ratings: {len(df.userID.to_list())}")

# Save the dataset for training in the second experiment
df.to_csv('./datasets/epinions/v1/ratings.csv', index=False)

In [4]:
df = pd.read_csv('./datasets/epinions/v1/ratings.csv')

In [5]:
# New dataframe
df.head()

,userID,itemID,rating
0,0,0,5
1,1,0,5
2,2,0,5
3,3,0,5
4,4,0,5


In [6]:
# Rating threshold for +ve and -ve ratings features
r_th = 4

# Get all the unique users
users_df = df[['userID']].drop_duplicates()

# Create the possible features
user_total_ratings = df.groupby(by="userID")["rating"].count()
user_positive_ratings = df[df['rating'] > r_th].groupby('userID')['rating'].count().reset_index()
user_negative_ratings = df[df['rating'] <= r_th].groupby('userID')['rating'].count().reset_index()

# Update main df
users_df["ratings"] = user_total_ratings.values
users_df = pd.merge(users_df, user_positive_ratings, on=['userID'], how='left').rename(columns={'rating':'positive_ratings'})
users_df = pd.merge(users_df, user_negative_ratings, on=['userID'], how='left').rename(columns={'rating':'negative_ratings'})

# # Clean none values
users_df.fillna(0, inplace=True)

In [7]:
import gower

# Get the gower distance matrix
distance_matrix = gower.gower_matrix(users_df.drop(columns=['userID']))

In [8]:
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=shc.ClusterWarning)
    # Compute linkage using the distance matrix
    linkage = shc.linkage(
        distance_matrix,
        method='ward'
    )

# Use fcluster to get the cluster labels
# `t` is the threshold to use to cut the dendrogram - higher `t` means less clusters / more data points within individual clusters
t = 6
clusters = shc.fcluster(linkage, t, criterion='distance')

# get unique cluster labels
unique_labels = np.unique(clusters)

# Adding the results to a new column in the dataframe
users_df["cluster_shc"] = clusters

print(f'Generated {len(unique_labels)} clusters.')

Generated 28 clusters.


In [9]:
counter = collections.Counter(users_df.cluster_shc.to_list())
print(counter)

Counter({2: 298, 1: 206, 5: 142, 12: 131, 3: 96, 21: 71, 23: 69, 10: 61, 4: 50, 8: 50, 11: 45, 14: 42, 9: 40, 7: 38, 22: 37, 13: 37, 6: 28, 20: 27, 28: 25, 26: 25, 27: 20, 17: 20, 24: 18, 16: 10, 25: 10, 18: 8, 19: 8, 15: 8})


## Visualize clusters (theoretical - Optional)

In [10]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Create a t-SNE object
tsne = TSNE(
    n_components=2,
    perplexity=30, # [5, 50] default is 30
    learning_rate=200, # [10.0, 1000.0] , def=200
    n_iter=1000, # >250, def=1000
    metric="euclidean"
)

# Perform t-SNE on the distance matrix
tsne_data = tsne.fit_transform(distance_matrix)
# test = tsne.fit(distance_matrix)

# Plot the t-SNE data using a scatter plot
plt.scatter(tsne_data[:, 0], tsne_data[:, 1])
plt.show()

C:\Users\wessa\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# Add the cluster labels to the t-SNE data
tsne_data_clusters = np.column_stack((tsne_data, clusters))

# Plot the t-SNE data using a scatter plot
plt.scatter(tsne_data_clusters[:, 0], tsne_data_clusters[:, 1], c=tsne_data_clusters[:, 2], cmap='Spectral')
plt.show()

# Create groups of the user clusters formed in the previous step

In [11]:
# Get the gower distance matrix
distance_matrix_2 = gower.gower_matrix(users_df.drop(columns=['userID']))

In [19]:
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=shc.ClusterWarning)
    # Compute linkage using the distance matrix
    linkage = shc.linkage(
        distance_matrix_2,
        method='ward'
    )

# Use fcluster to get the cluster labels
# `t` is the threshold to use to cut the dendrogram - higher `t` means less clusters / more data points within individual clusters
t = 4
clusters = shc.fcluster(linkage, t, criterion='distance')

# get unique cluster labels
unique_labels = np.unique(clusters)

# Adding the results to a new column in the dataframe
users_df["group_clusters"] = clusters

print(f'Generated {len(unique_labels)} clusters.')
counter = collections.Counter(users_df.group_clusters.to_list())
print(counter)

Generated 32 clusters.
Counter({22: 298, 21: 206, 19: 142, 32: 131, 20: 96, 5: 71, 31: 61, 18: 50, 25: 50, 30: 45, 2: 43, 26: 40, 27: 39, 1: 38, 23: 38, 28: 29, 24: 28, 4: 27, 3: 25, 12: 25, 17: 20, 13: 19, 11: 18, 14: 14, 15: 12, 29: 11, 16: 10, 10: 10, 7: 8, 6: 8, 9: 5, 8: 3})


In [16]:
# Save the clusters created
users_df.to_csv('./output/exp-3-epinions/clusters/clusters.csv', index=False)

# Clusters-Serendipity iterations experiment

In [20]:
import numpy as np
import scipy.cluster.hierarchy as shc
from models.lightgcn.train_clusters_script_epinions import train_on_groups

# Baseline serendipity calculated from a normal train in a previous experiment on the dataset
baseline_serendipity = pd.read_csv('./output/exp-3-epinions/baseline_serendipity.csv')

iterations = np.linspace(4, 5, 2)
for i in iterations:
    clusters = shc.fcluster(linkage, i, criterion='distance')
    # get unique cluster labels
    unique_labels = np.unique(clusters)

    # Total groups obtained (groups of clusters)
    print(len(unique_labels))

    # Save clusters in df
    users_df["group_clusters"] = clusters

    # Train model and check serendipity per group
    new_user_serendipity = train_on_groups(users_df)

    # Apply condition (if we increased serendipity per 0% of groups stop and save i)
    serendipity_df = baseline_serendipity.merge(new_user_serendipity, on='userID')
    serendipity_df['comparison'] = serendipity_df.apply(lambda x: 1 if (x.user_serendipity_y > x.user_serendipity_x) else 0, axis=1)
    total_users = len(serendipity_df)
    total_hits = sum(serendipity_df.comparison.to_list())

    threshold = total_hits/total_users * 100
    print(threshold)
    serendipity_df.to_csv('./output/exp-3-epinions/group_iterations_500k/' + str(i) + '.csv', index=False)

    if threshold > 80:
        print("Threshold achieved at", i)

32
Total groups: 32


C:\Users\wessa\AppData\Roaming\Python\Python39\site-packages\recommenders\models\deeprec\DataModel\ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)3.2s: train loss = 0.68202 = (mf)0.68193 + (embed)0.00009
Epoch 2 (train)1.1s: train loss = 0.61393 = (mf)0.61376 + (embed)0.00017
Epoch 3 (train)1.1s: train loss = 0.49318 = (mf)0.49286 + (embed)0.00032
Epoch 4 (train)1.0s: train loss = 0.41291 = (mf)0.41239 + (embed)0.00053
Epoch 5 (train)1.0s + (eval)2.1s: train loss = 0.36846 = (mf)0.36775 + (embed)0.00071, recall = 0.02056, ndcg = 0.17361, precision = 0.15263, map = 0.01018
Epoch 6 (train)1.0s: train loss = 0.34731 = (mf)0.34645 + (embed)0.00085
Epoch 7 (train)1.1s: train loss = 0.32685 = (mf)0.32588 + (embed)0.00097
Epoch 8 (train)1.0s: train loss = 0.30758 = (mf)0.30648 + (embed)0.00109
Epoch 9 (train)1.0s: train loss = 0.29791 = (mf)0.29669 + (embed)0.00122
Epoch 10 (train)1.0s + (eval)0.1s: train loss = 0.28150 = (mf)0.28015 + (embed)0.00135, recall = 0.02412, ndcg = 0.19253, precision = 0.17895, map = 0.01176
Epoch

C:\Users\wessa\AppData\Roaming\Python\Python39\site-packages\recommenders\models\deeprec\DataModel\ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)3.7s: train loss = 0.67368 = (mf)0.67359 + (embed)0.00009
Epoch 2 (train)1.6s: train loss = 0.56348 = (mf)0.56327 + (embed)0.00021
Epoch 3 (train)1.6s: train loss = 0.49160 = (mf)0.49120 + (embed)0.00040
Epoch 4 (train)1.6s: train loss = 0.46396 = (mf)0.46344 + (embed)0.00052
Epoch 5 (train)1.6s + (eval)2.1s: train loss = 0.42899 = (mf)0.42837 + (embed)0.00062, recall = 0.02255, ndcg = 0.20522, precision = 0.19767, map = 0.01041
Epoch 6 (train)1.5s: train loss = 0.39700 = (mf)0.39623 + (embed)0.00077
Epoch 7 (train)1.5s: train loss = 0.37388 = (mf)0.37293 + (embed)0.00095
Epoch 8 (train)1.6s: train loss = 0.35469 = (mf)0.35357 + (embed)0.00112
Epoch 9 (train)1.7s: train loss = 0.33948 = (mf)0.33818 + (embed)0.00130
Epoch 10 (train)1.6s + (eval)0.1s: train loss = 0.31790 = (mf)0.31642 + (embed)0.00148, recall = 0.02839, ndcg = 0.26026, precision = 0.25116, map = 0.01488
Epoch

...

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)8.4s: train loss = 0.68180 = (mf)0.68172 + (embed)0.00008
Epoch 2 (train)1.7s: train loss = 0.61051 = (mf)0.61036 + (embed)0.00016
Epoch 3 (train)1.7s: train loss = 0.49319 = (mf)0.49289 + (embed)0.00031
Epoch 4 (train)1.6s: train loss = 0.41526 = (mf)0.41476 + (embed)0.00050
Epoch 5 (train)1.7s + (eval)7.4s: train loss = 0.37100 = (mf)0.37033 + (embed)0.00067, recall = 0.02091, ndcg = 0.14426, precision = 0.13333, map = 0.00896
Epoch 6 (train)1.7s: train loss = 0.34697 = (mf)0.34616 + (embed)0.00081
Epoch 7 (train)1.7s: train loss = 0.32830 = (mf)0.32737 + (embed)0.00093
Epoch 8 (train)1.8s: train loss = 0.31137 = (mf)0.31033 + (embed)0.00105
Epoch 9 (train)1.8s: train loss = 0.29195 = (mf)0.29079 + (embed)0.00116
Epoch 10 (train)1.8s + (eval)0.2s: train loss = 0.27913 = (mf)0.27785 + (embed)0.00128, recall = 0.02667, ndcg = 0.17675, precision = 0.16889, map = 0.01177
Epoch

C:\Users\wessa\AppData\Roaming\Python\Python39\site-packages\recommenders\models\deeprec\DataModel\ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)8.8s: train loss = 0.68416 = (mf)0.68408 + (embed)0.00008
Epoch 2 (train)1.8s: train loss = 0.62304 = (mf)0.62288 + (embed)0.00016
Epoch 3 (train)1.8s: train loss = 0.50042 = (mf)0.50009 + (embed)0.00033
Epoch 4 (train)1.8s: train loss = 0.41412 = (mf)0.41356 + (embed)0.00056
Epoch 5 (train)1.8s + (eval)8.0s: train loss = 0.37084 = (mf)0.37007 + (embed)0.00076, recall = 0.02321, ndcg = 0.11807, precision = 0.12459, map = 0.00823
Epoch 6 (train)2.3s: train loss = 0.34697 = (mf)0.34605 + (embed)0.00092
Epoch 7 (train)2.0s: train loss = 0.33258 = (mf)0.33152 + (embed)0.00106
Epoch 8 (train)1.9s: train loss = 0.32193 = (mf)0.32074 + (embed)0.00119
Epoch 9 (train)2.0s: train loss = 0.31072 = (mf)0.30942 + (embed)0.00131
Epoch 10 (train)2.0s + (eval)0.2s: train loss = 0.29981 = (mf)0.29838 + (embed)0.00143, recall = 0.02363, ndcg = 0.13033, precision = 0.12623, map = 0.01000
Epoch

C:\Users\wessa\AppData\Roaming\Python\Python39\site-packages\recommenders\models\deeprec\DataModel\ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)10.8s: train loss = 0.66405 = (mf)0.66398 + (embed)0.00007
Epoch 2 (train)3.2s: train loss = 0.51121 = (mf)0.51095 + (embed)0.00026
Epoch 3 (train)3.2s: train loss = 0.44328 = (mf)0.44279 + (embed)0.00049
Epoch 4 (train)3.2s: train loss = 0.40289 = (mf)0.40225 + (embed)0.00063
Epoch 5 (train)3.2s + (eval)7.9s: train loss = 0.37589 = (mf)0.37510 + (embed)0.00079, recall = 0.02612, ndcg = 0.10741, precision = 0.10153, map = 0.01081
Epoch 6 (train)3.5s: train loss = 0.35781 = (mf)0.35687 + (embed)0.00094
Epoch 7 (train)3.4s: train loss = 0.34539 = (mf)0.34431 + (embed)0.00108
Epoch 8 (train)3.4s: train loss = 0.32983 = (mf)0.32862 + (embed)0.00121
Epoch 9 (train)3.4s: train loss = 0.31946 = (mf)0.31812 + (embed)0.00134
Epoch 10 (train)3.4s + (eval)0.3s: train loss = 0.31069 = (mf)0.30924 + (embed)0.00146, recall = 0.02894, ndcg = 0.11895, precision = 0.11450, map = 0.01253
Epoc

In [ ]:
users_df